In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf

from transformers import TFAutoModel, AutoTokenizer
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


# 1.TPU Configs

In [3]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  grpc://10.0.0.2:8470
REPLICAS:  8


# 2.Configuration

In [4]:
AUTO = tf.data.experimental.AUTOTUNE
# Configuration
EPOCHS = 2
BATCH_SIZE = 16 * strategy.num_replicas_in_sync
MAX_LEN = 80
MODEL = 'roberta-base'


In [5]:
def read_text_data(data_file):
    texts = []
    labels = []
    with open(data_file, "r") as f:
        while True:
            line = f.readline()
            if line:
                line = line.strip()
                text = line.split("\t")[1]
                label = line.split("\t")[0]
                texts.append(text)
                labels.append(label)
            else:    
                break
    return texts, labels

train_file = "/kaggle/input/relation-classification-kbp/kbp_relation_classification/train_sf3.txt"
test_file = "/kaggle/input/relation-classification-kbp/kbp_relation_classification/test_sf3.txt"

train_texts, train_labels = read_text_data(train_file)
test_texts, test_labels = read_text_data(test_file)

# label encode
le = LabelEncoder()
le.fit(train_labels)
train_labels = le.transform(train_labels)
test_labels = le.transform(test_labels)

train_labels = np.float32(train_labels)
test_labels = np.float32(test_labels)

print(len(train_texts))
print(len(test_labels))

28888
9600


# 3.encode token

In [6]:
# First load the real tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL)

def regular_encode(texts, tokenizer, maxlen=512):
    enc_di = tokenizer.batch_encode_plus(
        texts, 
        return_attention_masks=True, 
        return_token_type_ids=True,
        pad_to_max_length=True,
        max_length=maxlen
    )
    
    return np.array(enc_di['input_ids'], dtype=np.int32), np.array(enc_di['attention_mask'], dtype=np.int32), np.array(enc_di["token_type_ids"], dtype=np.int32)


# 4.Build model

In [7]:
def F1_scores(labels, predict):
    predict = np.argmax(predict, axis=1)
    result = f1_score(labels, predict, average="macro")
    return result
    
    
def build_model(transformer, max_len=512):
    """
    https://www.kaggle.com/xhlulu/jigsaw-tpu-distilbert-with-huggingface-and-keras
    """
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")
    sequence_output = transformer((input_word_ids, input_mask, segment_ids))[0]
    gp = tf.keras.layers.GlobalMaxPooling1D()(sequence_output)
    ap = tf.keras.layers.GlobalAveragePooling1D()(sequence_output)
    stack = tf.keras.layers.concatenate([gp, ap], axis=1)
    # stack = tf.keras.layers.Dropout(0.2)(stack)
    out = Dense(40, activation='softmax')(stack)
    
    
    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(Adam(lr=1e-6), loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])
    
    return model

# 5. Train

In [8]:
x_train = regular_encode(train_texts, tokenizer, maxlen=MAX_LEN)
x_test = regular_encode(test_texts, tokenizer, maxlen=MAX_LEN)

train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_train, train_labels))
    .repeat()
    .shuffle(50)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

test_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_test, test_labels))
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(AUTO)
)

In [9]:
%%time
with strategy.scope():
    transformer_layer = TFAutoModel.from_pretrained(MODEL)
    model = build_model(transformer_layer, max_len=MAX_LEN)
    
model.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 80)]         0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 80)]         0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 80)]         0                                            
__________________________________________________________________________________________________
tf_roberta_model (TFRobertaMode ((None, 80, 768), (N 124645632   input_word_ids[0][0]             
                                                                 input_mask[0][0]            

In [10]:
n_steps = len(train_texts) // BATCH_SIZE
train_history = model.fit(
    train_dataset,
    steps_per_epoch=n_steps,
    validation_data=test_dataset,
    epochs=70,
    shuffle=True,
)

test_predicts = model.predict(x_test)
result = F1_scores(test_labels, test_predicts)
print(result)

Epoch 1/60
225/225 [==============================] - 35s 154ms/step - accuracy: 0.0210 - loss: 4.2287 - val_accuracy: 0.1054 - val_loss: 3.5503
Epoch 2/60
225/225 [==============================] - 24s 106ms/step - accuracy: 0.1088 - loss: 3.6448 - val_accuracy: 0.1116 - val_loss: 3.3818
Epoch 3/60
225/225 [==============================] - 24s 105ms/step - accuracy: 0.1233 - loss: 3.4815 - val_accuracy: 0.1115 - val_loss: 3.3182
Epoch 4/60
225/225 [==============================] - 24s 107ms/step - accuracy: 0.1246 - loss: 3.4089 - val_accuracy: 0.1116 - val_loss: 3.2709
Epoch 5/60
225/225 [==============================] - 24s 107ms/step - accuracy: 0.1168 - loss: 3.3687 - val_accuracy: 0.1192 - val_loss: 3.2232
Epoch 6/60
225/225 [==============================] - 24s 105ms/step - accuracy: 0.1160 - loss: 3.3129 - val_accuracy: 0.1345 - val_loss: 3.1919
Epoch 7/60
225/225 [==============================] - 24s 106ms/step - accuracy: 0.1282 - loss: 3.2601 - val_accuracy: 0.1380 - va

225/225 [==============================] - 24s 106ms/step - accuracy: 0.7722 - loss: 0.6976 - val_accuracy: 0.7781 - val_loss: 0.6741
Epoch 58/60
225/225 [==============================] - 24s 107ms/step - accuracy: 0.7751 - loss: 0.6906 - val_accuracy: 0.7802 - val_loss: 0.6690
Epoch 59/60
225/225 [==============================] - 24s 107ms/step - accuracy: 0.7780 - loss: 0.6777 - val_accuracy: 0.7836 - val_loss: 0.6636
Epoch 60/60
225/225 [==============================] - 24s 107ms/step - accuracy: 0.7771 - loss: 0.6744 - val_accuracy: 0.7836 - val_loss: 0.6598
0.7272447435530616
